DeepWeeds Model

In [22]:
import os
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from PIL import Image
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

In [23]:
DATA_ROOT = "Deepweeds"
IMAGE_DIR = os.path.join(DATA_ROOT, "images")
LABEL_DIR = os.path.join(DATA_ROOT, "labels")

# Sanity checks
assert os.path.exists(IMAGE_DIR), f"Images directory not found: {IMAGE_DIR}"
assert os.path.exists(LABEL_DIR), f"Labels directory not found: {LABEL_DIR}"

FOLD = 0                  # choose 0–4 as per the files
FULL_FINETUNE = False     # False = partial, True = full
EPOCHS = 15
BATCH_SIZE = 32
LR_PARTIAL = 1e-3
LR_FULL = 1e-4
NUM_CLASSES = 9
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [24]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

eval_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [25]:
train_csv = os.path.join(LABEL_DIR, f"train_subset{FOLD}.csv")
val_csv   = os.path.join(LABEL_DIR, f"val_subset{FOLD}.csv")
test_csv  = os.path.join(LABEL_DIR, f"test_subset{FOLD}.csv")

assert os.path.exists(train_csv), "Train CSV not found"
assert os.path.exists(val_csv), "Validation CSV not found"
assert os.path.exists(test_csv), "Test CSV not found"

print("Using CSVs:")
print(train_csv)
print(val_csv)
print(test_csv)

Using CSVs:
Deepweeds/labels/train_subset0.csv
Deepweeds/labels/val_subset0.csv
Deepweeds/labels/test_subset0.csv


In [26]:
class DeepWeedsDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform):
        self.data = pd.read_csv(csv_file)
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        filename = self.data.iloc[idx]["Filename"]
        label = int(self.data.iloc[idx]["Label"])

        image_path = os.path.join(self.image_dir, filename)
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

In [27]:
train_dataset = DeepWeedsDataset(train_csv, IMAGE_DIR, train_transform)
val_dataset   = DeepWeedsDataset(val_csv, IMAGE_DIR, eval_transform)
test_dataset  = DeepWeedsDataset(test_csv, IMAGE_DIR, eval_transform)

print("Samples → Train:", len(train_dataset),
      "Val:", len(val_dataset),
      "Test:", len(test_dataset))


Samples → Train: 10501 Val: 3501 Test: 3507


In [28]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

In [29]:
def create_model(num_classes, full_finetune):
    weights = ResNet50_Weights.DEFAULT
    model = resnet50(weights=weights)

    """ FOR JUST FC TRAINING
    if not full_finetune:
        for param in model.parameters():
            param.requires_grad = False """

    """FOR PARTIAL UNFREEZING (LAST RESNET BLOCK) WITH FC TRAINING (by default)
    # Freeze entire backbone first
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze ONLY the last ResNet block (layer4)
    for param in model.layer4.parameters():
        param.requires_grad = True"""
    
    # FULL FINE-TUNING: unfreeze everything
    for param in model.parameters():
        param.requires_grad = True

    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

model = create_model(NUM_CLASSES, FULL_FINETUNE)
model = model.to(device)

print("Model on device:", next(model.parameters()).device)

Model on device: cuda:0


In [30]:
lr = LR_FULL if FULL_FINETUNE else LR_PARTIAL

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-4
)
loss_fn = nn.CrossEntropyLoss()

In [31]:
def train_one_epoch(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0.0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        logits = model(images)
        loss = loss_fn(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * images.size(0)

    return total_loss / len(loader.dataset)

In [32]:
def evaluate(model, loader):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            preds = model(images).argmax(dim=1)

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    p, r, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average="macro"
    )
    cm = confusion_matrix(y_true, y_pred)

    return acc, p, r, f1, cm

In [33]:
for epoch in range(EPOCHS):
    train_loss = train_one_epoch(model, train_loader, optimizer, loss_fn)
    val_acc, val_p, val_r, val_f1, _ = evaluate(model, val_loader)

    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"Train Loss: {train_loss:.4f} | "
        f"Val Acc: {val_acc:.4f} | "
        f"Val F1: {val_f1:.4f}"
    )

Epoch 1/15 | Train Loss: 1.0034 | Val Acc: 0.8375 | Val F1: 0.7834
Epoch 2/15 | Train Loss: 0.5093 | Val Acc: 0.8703 | Val F1: 0.8204
Epoch 3/15 | Train Loss: 0.4015 | Val Acc: 0.8883 | Val F1: 0.8564
Epoch 4/15 | Train Loss: 0.3388 | Val Acc: 0.8969 | Val F1: 0.8676
Epoch 5/15 | Train Loss: 0.2925 | Val Acc: 0.9306 | Val F1: 0.9114
Epoch 6/15 | Train Loss: 0.2652 | Val Acc: 0.9289 | Val F1: 0.9113
Epoch 7/15 | Train Loss: 0.2432 | Val Acc: 0.9403 | Val F1: 0.9239
Epoch 8/15 | Train Loss: 0.2289 | Val Acc: 0.9414 | Val F1: 0.9246
Epoch 9/15 | Train Loss: 0.2092 | Val Acc: 0.9360 | Val F1: 0.9184
Epoch 10/15 | Train Loss: 0.2006 | Val Acc: 0.9520 | Val F1: 0.9370
Epoch 11/15 | Train Loss: 0.1990 | Val Acc: 0.9420 | Val F1: 0.9266
Epoch 12/15 | Train Loss: 0.1758 | Val Acc: 0.9363 | Val F1: 0.9129
Epoch 13/15 | Train Loss: 0.1696 | Val Acc: 0.9414 | Val F1: 0.9243
Epoch 14/15 | Train Loss: 0.1713 | Val Acc: 0.9429 | Val F1: 0.9236
Epoch 15/15 | Train Loss: 0.1615 | Val Acc: 0.9429 | Val 

In [34]:
test_acc, test_p, test_r, test_f1, test_cm = evaluate(model, test_loader)

print("\nDeepWeeds Test Results")
print("Accuracy :", test_acc)
print("Precision:", test_p)
print("Recall   :", test_r)
print("F1-score :", test_f1)
print("Confusion Matrix:\n", test_cm)


DeepWeeds Test Results
Accuracy : 0.9404049044767607
Precision: 0.9391832607900044
Recall   : 0.9054789887953478
F1-score : 0.9195922945542416
Confusion Matrix:
 [[ 170    0    0    0    0    3    0   20   33]
 [   0  189    0    0    0    1    5    6   12]
 [   0    0  205    0    0    0    0    0    2]
 [   1    0    3  182    5    0    0    0   14]
 [   0    0   18    0  183    0    0    0   12]
 [   0    0    0    0    0  187    2    1   12]
 [   0    0    0    0    0    0  211    0    4]
 [   2    2    0    1    1    0    2  180   16]
 [   4    1    5    0    7    2   10    2 1791]]


On training just the classifier head  (ResNet frozen)   
(       for param in model.parameters():  
            param.requires_grad = False  
    model.fc = nn.Linear(model.fc.in_features, num_classes))  
  
lr = 1e-3  
  
Epoch 1/15 | Train Loss: 1.1900 | Val Acc: 0.6144 | Val F1: 0.3587  
Epoch 15/15 | Train Loss: 0.6840 | Val Acc: 0.7586 | Val F1: 0.6683
    
DeepWeeds Test Results  
Accuracy : 0.761619617907043  
Precision: 0.832071002990892  
Recall   : 0.5947460844495133  
F1-score : 0.6778236475786831  
Confusion Matrix:  
 [[ 111    5    0    4    2    3    1    7   93]  
 [   0  116    0    0    0    2    2    5   88]  
 [   0    0  142    1    9    0    0    0   55]  
 [   0    0    2   71    6    4    0    0  122]  
 [   0    0    3    3  143    1    0    0   63]  
 [   0    0    0    0    0  129    1    0   72]  
 [   0    2    0    0    0    2  128    2   81]  
 [  10    6    0    1    1    6    1   86   93]  
 [  12    7    2    8   16   22    5    5 1745]]  

On training partially frozen (resnet last block) along with fc layer  
lr = 1e-4    
  
Epoch 1/15 | Train Loss: 1.1141 | Val Acc: 0.7726 | Val F1: 0.6861  
Epoch 15/15 | Train Loss: 0.2407 | Val Acc: 0.9206 | Val F1: 0.8976  
  
DeepWeeds Test Results  
Accuracy : 0.9184488166524095  
Precision: 0.900410258967554  
Recall   : 0.8939264895038266  
F1-score : 0.8951552667033433  
Confusion Matrix:  
 [[ 174    4    0    7    0    0    1   19   21]  
 [   0  199    0    0    0    0    3    4    7]  
 [   1    0  201    3    2    0    0    0    0]  
 [   0    0    6  188    5    0    0    0    6]  
 [   0    0    5    7  192    0    0    0    9]  
 [   0    0    0    1    1  168    2    2   28]  
 [   0    0    0    0    0    0  205    0   10]  
 [   3    3    0    5    0    0    0  167   26]  
 [   6   15   12   11   13    7   21   10 1727]]  

On training fully resnet model with fc layer  
lr = 1e-4  
  
Epoch 1/15 | Train Loss: 1.0034 | Val Acc: 0.8375 | Val F1: 0.7834  
Epoch 15/15 | Train Loss: 0.1615 | Val Acc: 0.9429 | Val F1: 0.9258  
  
DeepWeeds Test Results  
Accuracy : 0.9404049044767607  
Precision: 0.9391832607900044  
Recall   : 0.9054789887953478  
F1-score : 0.9195922945542416  
Confusion Matrix:  
 [[ 170    0    0    0    0    3    0   20   33]  
 [   0  189    0    0    0    1    5    6   12]  
 [   0    0  205    0    0    0    0    0    2]  
 [   1    0    3  182    5    0    0    0   14]  
 [   0    0   18    0  183    0    0    0   12]  
 [   0    0    0    0    0  187    2    1   12]  
 [   0    0    0    0    0    0  211    0    4]  
 [   2    2    0    1    1    0    2  180   16]  
 [   4    1    5    0    7    2   10    2 1791]]  